### Related Course

In [16]:
import numpy as np
import pandas as pd
import joblib
from tensorflow.keras.models import load_model

def recommend_courses(user_id, interaction_matrix_path='./CF-Autoencoder/interaction_matrix.npy',
                      course_data_path='udemy_courses (1).csv',
                      model_path='./CF-Autoencoder/autoencoder_model.keras',
                      user_encoder_path='./CF-Autoencoder/user_encoder.pkl',
                      course_encoder_path='./CF-Autoencoder/course_encoder.pkl',
                      top_n=5):
    # Load model and encoders
    try:
        autoencoder = load_model(model_path)
        user_encoder = joblib.load(user_encoder_path)
        course_encoder = joblib.load(course_encoder_path)
        courses_df = pd.read_csv(course_data_path)
        interaction_matrix = np.load(interaction_matrix_path)
    except Exception as e:
        print(f"❌ Error loading model or encoders: {e}")
        return []

    # Check if user exists
    if user_id not in user_encoder.classes_:
        print("❌ User not found in encoder.")
        return []
    # Ensure binary matrix (optional based on training logic)
    interaction_matrix = (interaction_matrix > 0).astype(int)

    # Get user index
    user_idx = user_encoder.transform([user_id])[0]

    # Predict course scores (reshape to avoid retracing warnings)
    try:
        user_input = interaction_matrix[user_idx].reshape(1, -1)
        preds = autoencoder.predict(user_input, verbose=0)[0]
    except Exception as e:
        print(f"❌ Error during prediction: {e}")
        return []

    # Courses already taken
    already_taken = set(np.where(interaction_matrix[user_idx] > 0)[0])
    print(f"🧠 Already taken: {len(already_taken)} courses")

    # Rank predictions
    top_indices = np.argsort(preds)[::-1]
    # Filter already taken and get top_n
    recommended_indices = []
    for idx in top_indices:
        recommended_indices.append(idx)

    if len(recommended_indices) < top_n:
        print(f"⚠️ Only {len(recommended_indices)} unseen recommendations found (requested {top_n})")

    # Decode course IDs
    try:
        recommended_course_ids = course_encoder.inverse_transform(recommended_indices)
    except Exception as e:
        print(f"❌ Error decoding course IDs: {e}")
        return []

    return recommended_course_ids.tolist()


In [1]:
import pandas as pd 
df= pd.read_csv('./udemy_courses (1).csv')

In [4]:
df['id'].value_counts()

id
567828     1
387320     1
444004     1
2661384    1
275992     1
          ..
1130954    1
847476     1
4287908    1
5277072    1
2341728    1
Name: count, Length: 98104, dtype: int64

In [ ]:

# Recommendation function (copied from your input)
def recommend_courses(user_id, interaction_matrix_path='./CF-Autoencoder/interaction_matrix.npy',
                      course_data_path='udemy_courses (1).csv',
                      model_path='./CF-Autoencoder/autoencoder_model.keras',
                      user_encoder_path='./CF-Autoencoder/user_encoder.pkl',
                      course_encoder_path='./CF-Autoencoder/course_encoder.pkl',
                      top_n=5):
    try:
        autoencoder = load_model(model_path)
        user_encoder = joblib.load(user_encoder_path)
        course_encoder = joblib.load(course_encoder_path)
        courses_df = pd.read_csv(course_data_path)
        interaction_matrix = np.load(interaction_matrix_path)
    except Exception as e:
        return {"error": f"Error loading model or encoders: {e}"}

    if user_id not in user_encoder.classes_:
        return {"error": "User not found in encoder."}

    interaction_matrix = (interaction_matrix > 0).astype(int)
    user_idx = user_encoder.transform([user_id])[0]

    try:
        user_input = interaction_matrix[user_idx].reshape(1, -1)
        preds = autoencoder.predict(user_input, verbose=0)[0]
    except Exception as e:
        return {"error": f"Error during prediction: {e}"}

    already_taken = set(np.where(interaction_matrix[user_idx] > 0)[0])
    top_indices = np.argsort(preds)[::-1]

    recommended_indices = [idx for idx in top_indices if idx not in already_taken][:top_n]

    if not recommended_indices:
        return {"warning": "No new recommendations found."}

    try:
        recommended_course_ids = course_encoder.inverse_transform(recommended_indices)
    except Exception as e:
        return {"error": f"Error decoding course IDs: {e}"}

    return {
        "user_id": user_id,
        "recommended_courses": recommended_course_ids.tolist()
    }

# Flask endpoint
@app.route('/recommend-cf', methods=['GET'])
def recommend():
    user_id = request.args.get('user_id')
    if not user_id:
        return jsonify({"error": "Missing user_id parameter"}), 400

    result = recommend_courses(user_id)
    return jsonify(result)
